In [1]:
import json

In [2]:
with open('documents.json', 'rt') as f:
    docs_raw= json.load(f)

In [5]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
q = 'can I enroll the course even it has already started?'

In [13]:
def search(query, doc):
    boost = {'question': 3.0, 'section': .5}

    result = index.search(
        query = query,
        filter_dict={'course':'data-engineering-zoomcamp'},
        boost_dict= boost,
        num_results= 10 
    )

    return result

In [14]:
def build_prompt(query, result):

    prompt_temp = '''
    you're a course teaching assistant. Answer the QUESTION based on the CONTEXT.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT:
    {context}
    '''.strip()
    
    context = ""

    for doc in result:
        context = context + f"section: {doc['section']}\nquestions: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_temp.format(question = query, context = context).strip()

    return prompt

In [24]:
def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [17]:
query = 'if the course has already started, can i enroll the course?'

In [18]:
from elasticsearch import Elasticsearch

In [19]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.4/360.4 kB 9.2 MB/s eta 0:00:000m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 9.3 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [20]:
from openai import OpenAI

In [22]:
client = OpenAI(
    base_url = 'http://localhost:11434/v1/',
    api_key = 'ollama',
)

In [32]:
def llm(prompt):
    response = client.chat.completions.create(
        model = 'phi3',
        messages= [
            {
                'role':'user',
                'content': prompt
            }
            
        ]
    )
    return response.choices[0].message.content

In [25]:
es_client = Elasticsearch('http://localhost:9200')

In [26]:
index_setting = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

In [27]:
index_name = "course-questions"

In [28]:
es_client.indices.create(index=index_name, body= index_setting)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [29]:
for doc in documents:
    es_client.index(index=index_name, document=doc)

In [30]:
def elastics_search(query):

    search_query ={
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    # search
    response = es_client.search(index= index_name, body= search_query)

    result_docs = []

    for hits in response['hits']['hits']:
        result_docs.append(hits['_source'])


    return result_docs

    
    
    

In [34]:
def rag_es(query, doc):

    search_result = elastics_search(query)
    prompt = build_prompt(query, search_result)
    answer = llm(prompt)

    return answer
    

In [35]:
rag_es('Can i enroll the course after it started', doc)

InternalServerError: Error code: 500 - {'error': {'message': 'model requires more system memory (5.6 GiB) than is available (4.8 GiB)', 'type': 'api_error', 'param': None, 'code': None}}